# SQL Challenge 2 Notebook

## Data Cleaning

In [ ]:
--Fix inconsistent null values in customer_orders
UPDATE pizza_runner.customer_orders
	SET exclusions = NULLIF(NULLIF(NULLIF(exclusions, ''), 'null'), 'NaN'),
    	extras = NULLIF(NULLIF(NULLIF(extras, ''), 'null'), 'NaN');
    
-- remove unnecessary text in distance and duration measurements and fix inconsistent nulls throughout.
ALTER TABLE pizza_runner.runner_orders
	ADD COLUMN pickup_time_clean TIMESTAMP,
    ADD COLUMN distance_clean NUMERIC(5,2),
    ADD COLUMN duration_clean INTEGER,
    ADD COLUMN cancellation_clean TEXT;

UPDATE pizza_runner.runner_orders
	SET pickup_time_clean = NULLIF(pickup_time, 'null')::TIMESTAMP,
    	distance_clean = CASE
        	WHEN distance LIKE '%km' THEN TRIM(REPLACE(distance, 'km', ''))::NUMERIC
            ELSE NULLIF(distance, 'null')::NUMERIC
            END,
       	duration_clean = CASE
        	WHEN duration ~ '\d+' THEN TRIM(REGEXP_REPLACE(duration, '[^\d]', '', 'g'))::INTEGER
            ELSE NULL
            END,
     	cancellation_clean = NULLIF(NULLIF(NULLIF(cancellation, ''), 'NaN'), 'null')::TEXT;
        
ALTER TABLE pizza_runner.runner_orders
 	DROP COLUMN pickup_time,
    DROP COLUMN distance,
    DROP COLUMN duration,
    DROP COLUMN cancellation;
 
ALTER TABLE pizza_runner.runner_orders
 	RENAME COLUMN pickup_time_clean TO pickup_time;

ALTER TABLE pizza_runner.runner_orders
 	RENAME COLUMN distance_clean TO distance;

ALTER TABLE pizza_runner.runner_orders
 	RENAME COLUMN duration_clean TO duration;

ALTER TABLE pizza_runner.runner_orders
 	RENAME COLUMN cancellation_clean TO cancellation;

## Questions
### Pizza Metrics
#### 1. How many pizzas were ordered?

In [ ]:
SELECT COUNT(*) FROM pizza_runner.customer_orders;

| count |
| ----- |
| 14    |
#### 2. How many unique customer orders were made?

In [ ]:
SELECT COUNT(DISTINCT (order_id, customer_id, pizza_id, exclusions, extras, order_time))
FROM pizza_runner.customer_orders;

| count |
| ----- |
| 13    |
#### 3. How many successful orders were delivered by each runner?

In [ ]:
SELECT COUNT(*)
FROM pizza_runner.runner_orders
WHERE cancellation ISNULL;

| count |
| ----- |
| 8     |


#### 4. How many of each type of pizza was delivered? 

In [ ]:
SELECT pizza_id, COUNT(*)
FROM pizza_runner.customer_orders c
JOIN pizza_runner.runner_orders r
	ON r.order_id = c.order_id
WHERE cancellation IS NULL
GROUP BY pizza_id;

| pizza_id | count |
| -------- | ----- |
| 1        | 9     |
| 2        | 3     |
#### 5. How many Vegetarian and Meatlovers were ordered by each customer?

In [ ]:
SELECT customer_id, pizza_name, COUNT(*)
FROM pizza_runner.customer_orders c
JOIN pizza_runner.pizza_names p
	ON c.pizza_id = p.pizza_id
GROUP BY customer_id, pizza_name
ORDER BY customer_id ASC;

| customer_id | pizza_name | count |
| ----------- | ---------- | ----- |
| 101         | Meatlovers | 2     |
| 101         | Vegetarian | 1     |
| 102         | Meatlovers | 2     |
| 102         | Vegetarian | 1     |
| 103         | Meatlovers | 3     |
| 103         | Vegetarian | 1     |
| 104         | Meatlovers | 3     |
| 105         | Vegetarian | 1     |
#### 6. What was the maximum number of pizzas delivered in a single order?

In [ ]:
SELECT MAX(pizza_count) AS max_pizzas_at_once
FROM (
  SELECT order_time, COUNT(*) AS pizza_count
  FROM pizza_runner.customer_orders
  GROUP BY order_time
  ) AS t;

| max_pizzas_at_once |
| ------------------ |
| 3                  |
#### 7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?

In [ ]:
SELECT pizza_id, COUNT(*)
FROM pizza_runner.customer_orders c
JOIN pizza_runner.runner_orders r
	ON r.order_id = c.order_id
WHERE cancellation IS NULL
GROUP BY pizza_id;

| without_changes | with_changes |
| --------------- | ------------ |
| 7               | 7            |
#### 8. How many pizzas were delivered that had both exclusions and extras?

In [ ]:
SELECT COUNT(*) AS exclusions_and_extras
FROM pizza_runner.customer_orders c
JOIN pizza_runner.runner_orders r
	ON c.order_id = r.order_id
WHERE r.cancellation IS NULL
	AND exclusions IS NOT NULL
    AND extras IS NOT NULL;

| exclusions_and_extras |
| --------------------- |
| 1                     |
#### 9. What was the total volume of pizzas ordered for each hour of the day?

In [ ]:
SELECT EXTRACT(HOUR FROM order_time)::INTEGER AS time_of_day,
	COUNT(*) AS pizzas_ordered
FROM pizza_runner.customer_orders
GROUP BY time_of_day
ORDER BY time_of_day;


#### 10. What was the volume of orders for each day of the week?

In [ ]:
SELECT EXTRACT(DOW FROM order_time)::INTEGER AS day_of_week,
	COUNT(*) AS pizzas_ordered
FROM pizza_runner.customer_orders
GROUP BY day_of_week
ORDER BY day_of_week;

| day_of_week | pizzas_ordered |
| ----------- | -------------- |
| 3           | 5              |
| 4           | 3              |
| 5           | 1              |
| 6           | 5              |
### Runner and Customer Experience
#### 1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [ ]:
SELECT (registration_date::date - ((registration_date::date - DATE '2021-01-01') % 7)) AS week_start, COUNT(*) AS runner_signups
FROM runners
GROUP BY week_start
ORDER BY week_start ASC;

| week_start | runner_signups |
| ---------- | -------------- |
| 2021-01-01 | 2              |
| 2021-01-08 | 1              |
| 2021-01-15 | 1              |
#### 2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

In [ ]:
SELECT
	ROUND(AVG(EXTRACT(EPOCH FROM pickup_time - order_time)/60)) AS avg_pickup_interval
FROM customer_orders c
JOIN runner_orders r
	ON c.order_id = r.order_id;

| avg_pickup_interval |
| ------------------- |
| 19                  |
#### 3. Is there any relationship between the number of pizzas and how long the order takes to prepare?
##### ASSUMPTION: 
Pizzas are picked up as soon as they are prepared. If this assumption is not true, the data has to contain a timestamp for when the pizzas are actually prepped.

In [ ]:
WITH order_stats AS (
  SELECT
    c.order_id,
    COUNT(*) AS num_pizzas,
    MAX(r.pickup_time)  
      - MIN(c.order_time) AS prep_time
  FROM pizza_runner.customer_orders c
  JOIN pizza_runner.runner_orders r
    ON c.order_id = r.order_id
  WHERE r.pickup_time IS NOT NULL     -- only delivered orders
  GROUP BY c.order_id
)
SELECT
  num_pizzas,
  AVG(prep_time) AS avg_prep_time
FROM order_stats
GROUP BY num_pizzas
ORDER BY num_pizzas;

| num_pizzas | avg_prep_time | median_prep_time |
| ---------- | ------------- | ---------------- |
| 1          | 00:12:21.4    | 00:10:28         |
| 2          | 00:18:22.5    | 00:18:22.5       |
| 3          | 00:29:17      | 00:29:17         |

Yes, orders with more pizzas tend to take significantly longer to prepare. 
#### 4. What was the average distance travelled for each customer?

In [ ]:
SELECT 
	c.customer_id,
  	ROUND(AVG(r.distance), 2)
FROM pizza_runner.customer_orders c
JOIN pizza_runner.runner_orders r
	ON c.order_id = r.order_id
WHERE r.cancellation IS NULL
GROUP BY c.customer_id
ORDER BY c.customer_id;

| customer_id | round |
| ----------- | ----- |
| 101         | 20.00 |
| 102         | 16.73 |
| 103         | 23.40 |
| 104         | 10.00 |
| 105         | 25.00 |
#### 5. What was the difference between the longest and shortest delivery times for all orders?

In [ ]:
SELECT
	MAX(duration) - MIN(duration) AS duration_range
FROM pizza_runner.runner_orders;

| duration_range |
| -------------- |
| 30             |
#### 6. What was the average speed for each runner and do you notice any trend for these values?

In [ ]:
SELECT 
	runner_id, 
    ROUND(SUM(distance) / SUM(duration), 2) AS average_rate
FROM pizza_runner.runner_orders
WHERE cancellation IS NULL
GROUP BY runner_id;

| runner_id | average_rate |
| --------- | ------------ |
| 1         | 0.71         |
| 2         | 0.90         |
| 3         | 0.67         |

Runner 2 has a significantly higher average rate than the other runners.
#### 7. What is the successful delivery percentage for each runner?

In [ ]:
SELECT
  runner_id,
  format(
    '%s%%',
    ROUND(AVG((cancellation IS NULL)::int) * 100, 2)
  ) AS success_rate
FROM pizza_runner.runner_orders
GROUP BY runner_id
ORDER BY runner_id;

| runner_id | success_rate |
| --------- | ------------ |
| 1         | 100.00%      |
| 2         | 75.00%       |
| 3         | 50.00%       |
### Ingredient Optimization
#### 1. What are the standard ingredients for each pizza?

In [ ]:
SELECT
	p.pizza_name,
    t.topping_name
FROM pizza_runner.pizza_names p
JOIN pizza_runner.pizza_recipes pr
	ON p.pizza_id = pr.pizza_id
JOIN LATERAL unnest(string_to_array(pr.toppings, ',')) AS tid(topping_id_text)
	ON TRUE
JOIN pizza_runner.pizza_toppings t
	ON t.topping_id = tid.topping_id_text::INT
ORDER BY p.pizza_name, t.topping_name;

| pizza_name | topping_name |
| ---------- | ------------ |
| Meatlovers | BBQ Sauce    |
| Meatlovers | Bacon        |
| Meatlovers | Beef         |
| Meatlovers | Cheese       |
| Meatlovers | Chicken      |
| Meatlovers | Mushrooms    |
| Meatlovers | Pepperoni    |
| Meatlovers | Salami       |
| Vegetarian | Cheese       |
| Vegetarian | Mushrooms    |
| Vegetarian | Onions       |
| Vegetarian | Peppers      |
| Vegetarian | Tomato Sauce |
| Vegetarian | Tomatoes     |
#### 2. What was the most commonly added extra?

In [ ]:
SELECT 
	pt.topping_name,
    COUNT(*) AS count
FROM pizza_runner.customer_orders c
JOIN LATERAL unnest(
	string_to_array(
		c.extras, ',')
	) AS tid(topping_id_text)
	ON TRUE
JOIN pizza_runner.pizza_toppings pt
	ON pt.topping_id = tid.topping_id_text::INT
GROUP BY pt.topping_name
ORDER BY count DESC
LIMIT 1;

| topping_name | count |
| ------------ | ----- |
| Bacon        | 4     |
#### 3. What was the most common exclusion?

In [ ]:
SELECT
	t.topping_name,
    COUNT(*) AS count
FROM pizza_runner.customer_orders c
JOIN LATERAL unnest(
    string_to_array(
		c.exclusions, ',')
	) AS tid(topping_id_text)
	ON TRUE
JOIN pizza_runner.pizza_toppings t
	ON t.topping_id = tid.topping_id_text::INT
GROUP BY t.topping_name
ORDER BY count DESC
LIMIT 1;

| topping_name | count |
| ------------ | ----- |
| Cheese       | 4     |
#### 4. Generate an order item for each record in the customers_orders table in the format of one of the following:

    Meat Lovers
    Meat Lovers - Exclude Beef
    Meat Lovers - Extra Bacon
    Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers


In [ ]:
WITH excl_list AS (
  	SELECT
  		c.order_line_id,
  		string_agg(
          	pt.topping_name,
          	', ' 
          	ORDER BY pt.topping_name
        ) AS excl_list
  	FROM pizza_runner.customer_orders c
  	JOIN LATERAL unnest(
      	string_to_array(c.exclusions, ',')
    ) AS tid(topping_text)
    	ON TRUE
  	JOIN pizza_runner.pizza_toppings pt
  		ON pt.topping_id = tid.topping_text::INT
    GROUP BY c.order_line_id
), 
extr_list AS (
  	SELECT 
  		c.order_line_id,
  		string_agg(
          	pt.topping_name, 
          	', ' 
          	ORDER BY pt.topping_name
        ) AS extr_list
  	FROM pizza_runner.customer_orders c
  	JOIN LATERAL unnest(
      	string_to_array(c.extras, ',')
    ) AS tid(topping_id_text)
  		ON TRUE
  	JOIN pizza_runner.pizza_toppings pt
  		ON pt.topping_id = tid.topping_id_text::INT
  	GROUP BY c.order_line_id
)

SELECT
	c.order_line_id,
    pn.pizza_name,
    COALESCE(excl.excl_list, '') AS exclusions_list,
    COALESCE(extr.extr_list, '') AS extras_list,
    CONCAT_WS(
      	' - ',
      	pn.pizza_name,
      	NULLIF('Exclude ' || excl.excl_list, 'Exclude '),
      	NULLIF('Extra ' || extr.extr_list, 'Extra ')
    ) AS order_description
FROM pizza_runner.customer_orders c
JOIN pizza_runner.pizza_names pn
	ON c.pizza_id = pn.pizza_id
LEFT JOIN excl_list excl
	ON c.order_line_id = excl.order_line_id
LEFT JOIN extr_list extr
	ON c.order_line_id = extr.order_line_id
ORDER BY order_line_id;

| order_line_id | pizza_name | exclusions_list      | extras_list    | order_description                                               |
| ------------- | ---------- | -------------------- | -------------- | --------------------------------------------------------------- |
| 1             | Meatlovers |                      |                | Meatlovers                                                      |
| 2             | Meatlovers |                      |                | Meatlovers                                                      |
| 3             | Meatlovers |                      |                | Meatlovers                                                      |
| 4             | Vegetarian |                      |                | Vegetarian                                                      |
| 5             | Meatlovers | Cheese               |                | Meatlovers - Exclude Cheese                                     |
| 6             | Meatlovers | Cheese               |                | Meatlovers - Exclude Cheese                                     |
| 7             | Vegetarian | Cheese               |                | Vegetarian - Exclude Cheese                                     |
| 8             | Meatlovers |                      | Bacon          | Meatlovers - Extra Bacon                                        |
| 9             | Vegetarian |                      |                | Vegetarian                                                      |
| 10            | Vegetarian |                      | Bacon          | Vegetarian - Extra Bacon                                        |
| 11            | Meatlovers |                      |                | Meatlovers                                                      |
| 12            | Meatlovers | Cheese               | Bacon, Chicken | Meatlovers - Exclude Cheese - Extra Bacon, Chicken              |
| 13            | Meatlovers |                      |                | Meatlovers                                                      |
| 14            | Meatlovers | BBQ Sauce, Mushrooms | Bacon, Cheese  | Meatlovers - Exclude BBQ Sauce, Mushrooms - Extra Bacon, Cheese |
#### 5. Generate an alphabetically ordered comma separated ingredient list for each pizza order from the customer_orders table and add a 2x in front of any relevant ingredients

    For example: "Meat Lovers: 2xBacon, Beef, ... , Salami"

In [ ]:
WITH parsed AS (
  	SELECT 
  		c.order_line_id,
  		pn.pizza_name,
  		COALESCE(string_to_array(pr.toppings, ',')::integer[], '{}'::integer[])
			AS recipe_tops,
  		COALESCE(string_to_array(c.exclusions, ',')::integer[], '{}'::integer[])
  			AS excl_tops,
  		COALESCE(string_to_array(c.extras, ',')::integer[], '{}'::integer[])
  			AS extra_tops
  	FROM pizza_runner.customer_orders c
    JOIN pizza_runner.pizza_names pn
    	ON c.pizza_id = pn.pizza_id
    JOIN pizza_runner.pizza_recipes pr
    	ON c.pizza_id = pr.pizza_id
),
recipe_exploded AS (
	SELECT
    	order_line_id,
        pizza_name,
        rid.topping_id,
        extra_tops,
  		TRUE AS is_recipe
   	FROM parsed
    CROSS JOIN LATERAL unnest(parsed.recipe_tops)
    	AS rid(topping_id)
    WHERE NOT (rid.topping_id = ANY(parsed.excl_tops))
),
extras_only AS (
  	SELECT
  		order_line_id,
  		pizza_name,
  		ext.topping_id,
  		extra_tops,
        FALSE AS is_recipe
  	FROM parsed
  	CROSS JOIN LATERAL unnest(parsed.extra_tops)
  		AS ext(topping_id)
  	WHERE NOT (ext.topping_id = ANY(parsed.recipe_tops))
),
all_toppings AS (
  	SELECT * FROM recipe_exploded
  	UNION ALL
  	SELECT * FROM extras_only
)

SELECT
	a.order_line_id,
    a.pizza_name,
    string_agg(
      	CASE
      		WHEN a.is_recipe
      				AND a.topping_id = ANY(a.extra_tops)
				THEN '2x ' || pt.topping_name
      		ELSE pt.topping_name
      	END,
      	', ' ORDER BY pt.topping_name
    ) AS ingredient_list
FROM all_toppings a
JOIN pizza_runner.pizza_toppings pt
	ON pt.topping_id = a.topping_id
GROUP BY a.order_line_id, a.pizza_name
ORDER BY a.order_line_id, a.pizza_name;

| order_line_id | pizza_name | ingredient_list                                                          |
| ------------- | ---------- | ------------------------------------------------------------------------ |
| 1             | Meatlovers | BBQ Sauce, Bacon, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami    |
| 2             | Meatlovers | BBQ Sauce, Bacon, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami    |
| 3             | Meatlovers | BBQ Sauce, Bacon, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami    |
| 4             | Vegetarian | Cheese, Mushrooms, Onions, Peppers, Tomato Sauce, Tomatoes               |
| 5             | Meatlovers | BBQ Sauce, Bacon, Beef, Chicken, Mushrooms, Pepperoni, Salami            |
| 6             | Meatlovers | BBQ Sauce, Bacon, Beef, Chicken, Mushrooms, Pepperoni, Salami            |
| 7             | Vegetarian | Mushrooms, Onions, Peppers, Tomato Sauce, Tomatoes                       |
| 8             | Meatlovers | BBQ Sauce, 2x Bacon, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami |
| 9             | Vegetarian | Cheese, Mushrooms, Onions, Peppers, Tomato Sauce, Tomatoes               |
| 10            | Vegetarian | Bacon, Cheese, Mushrooms, Onions, Peppers, Tomato Sauce, Tomatoes        |
| 11            | Meatlovers | BBQ Sauce, Bacon, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami    |
| 12            | Meatlovers | BBQ Sauce, 2x Bacon, Beef, 2x Chicken, Mushrooms, Pepperoni, Salami      |
| 13            | Meatlovers | BBQ Sauce, Bacon, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami    |
| 14            | Meatlovers | 2x Bacon, Beef, 2x Cheese, Chicken, Pepperoni, Salami                    |
#### 6. What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?

In [ ]:
WITH parsed AS (
  	SELECT
  		c.order_line_id,
  		COALESCE(string_to_array(pr.toppings, ',')::integer[],
                 '{}'::integer[]) AS recipe_tops,
  		COALESCE(string_to_array(c.exclusions, ',')::integer[],
                 '{}'::integer[]) AS excl_tops,
  		COALESCE(string_to_array(c.extras, ',')::integer[],
                 '{}'::integer[]) AS extra_tops
  	FROM pizza_runner.customer_orders c
  	JOIN pizza_runner.pizza_recipes pr
  		ON pr.pizza_id = c.pizza_id
  	JOIN pizza_runner.runner_orders r
  		ON c.order_id = r.order_id
  	WHERE r.cancellation IS NULL
),
all_toppings AS (
    SELECT
        order_line_id,
        rid.topping_id
    FROM parsed
    CROSS JOIN LATERAL unnest(parsed.recipe_tops)
    	AS rid(topping_id)
    WHERE NOT (rid.topping_id = ANY(parsed.excl_tops))
    UNION ALL
	SELECT
  		order_line_id,
  		ext.topping_id
  	FROM parsed
  	CROSS JOIN LATERAL unnest(parsed.extra_tops)
  		AS ext(topping_id)
)
SELECT
	pt.topping_name,
    COUNT(*) quantity
FROM all_toppings a
JOIN pizza_runner.pizza_toppings pt
	ON a.topping_id = pt.topping_id
GROUP BY pt.topping_name
ORDER BY quantity DESC;

| topping_name | quantity |
| ------------ | -------- |
| Bacon        | 12       |
| Mushrooms    | 11       |
| Cheese       | 10       |
| Pepperoni    | 9        |
| Chicken      | 9        |
| Salami       | 9        |
| Beef         | 9        |
| BBQ Sauce    | 8        |
| Tomato Sauce | 3        |
| Onions       | 3        |
| Tomatoes     | 3        |
| Peppers      | 3        |
### Pricing and Ratings
#### 1. If a Meat Lovers pizza costs \\$12 and Vegetarian costs \\$10 and there were no charges for changes - how much money has Pizza Runner made so far if there are no delivery fees?

In [ ]:
WITH prices as (
  	VALUES (1, 12), (2, 10)
)
SELECT '$' || SUM(price) AS revenue
FROM (
  	SELECT
  		c.pizza_id
  	FROM pizza_runner.customer_orders c
  	JOIN pizza_runner.runner_orders r USING(order_id)
  	WHERE r.cancellation IS NULL
) AS sub
JOIN prices AS p(pizza_id,price) USING(pizza_id);

| revenue |
| ------- |
| $138    |
#### 2. What if there was an additional $1 charge for any pizza extras?

In [ ]:
WITH prices as (
  	VALUES (1, 12), (2, 10)
)
SELECT '$' || SUM(price +
           extra_cnt
          ) AS revenue
FROM (
  	SELECT
  		pizza_id,
  		COALESCE(array_length(string_to_array(extras, ',')::integer[], 1),
                 0) AS extra_cnt
  	FROM pizza_runner.customer_orders c
  	JOIN pizza_runner.runner_orders r USING(order_id)
  	WHERE r.cancellation IS NULL
) AS sub
JOIN prices AS p(pizza_id, price) USING(pizza_id);

| revenue |
| ------- |
| $142    |
#### 3. The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, how would you design an additional table for this new dataset - generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.

In [ ]:
DROP TABLE IF EXISTS runner_ratings;
CREATE TABLE runner_ratings (
  "order_id" INTEGER,
  "rating" INTEGER
);
INSERT INTO runner_ratings
  ("order_id", "rating")
VALUES
  (1, 5),
  (2, 4),
  (3, 5),
  (4, 2),
  (5, 4),
  (7, 4),
  (8, 5),
  (10, 3);

#### 4. Using your newly generated table - can you join all of the information together to form a table which has the following information for successful deliveries?

    customer_id
    order_id
    runner_id
    rating
    order_time
    pickup_time
    Time between order and pickup
    Delivery duration
    Average speed
    Total number of pizzas

In [ ]:
WITH order_lines AS (
  	SELECT 
  		order_id,
  		COUNT(*)
  	FROM pizza_runner.customer_orders c
  	JOIN pizza_runner.runner_orders r USING(order_id)
  	WHERE r.cancellation IS NULL
  	GROUP BY order_id
)
SELECT
	customer_id,
    order_id,
    runner_id,
    rating,
    order_time,
    pickup_time,
    pickup_time - order_time AS prep_time,
    duration,
    ROUND(distance / duration, 2) AS avg_speed,
    o.count AS pizza_qty
FROM pizza_runner.customer_orders c
JOIN pizza_runner.runner_orders r USING(order_id)
JOIN order_lines o USING(order_id)
JOIN pizza_runner.runner_ratings USING(order_id)
WHERE r.cancellation IS NULL;

| customer_id | order_id | runner_id | rating | order_time          | pickup_time         | prep_time | duration | avg_speed | pizza_qty |
| ----------- | -------- | --------- | ------ | ------------------- | ------------------- | --------- | -------- | --------- | --------- |
| 101         | 1        | 1         | 5      | 2020-01-01 18:05:02 | 2020-01-01 18:15:34 | 00:10:32  | 32       | 0.63      | 1         |
| 101         | 2        | 1         | 4      | 2020-01-01 19:00:52 | 2020-01-01 19:10:54 | 00:10:02  | 27       | 0.74      | 1         |
| 102         | 3        | 1         | 5      | 2020-01-02 23:51:23 | 2020-01-03 00:12:37 | 00:21:14  | 20       | 0.67      | 2         |
| 102         | 3        | 1         | 5      | 2020-01-02 23:51:23 | 2020-01-03 00:12:37 | 00:21:14  | 20       | 0.67      | 2         |
| 103         | 4        | 2         | 2      | 2020-01-04 13:23:46 | 2020-01-04 13:53:03 | 00:29:17  | 40       | 0.59      | 3         |
| 103         | 4        | 2         | 2      | 2020-01-04 13:23:46 | 2020-01-04 13:53:03 | 00:29:17  | 40       | 0.59      | 3         |
| 103         | 4        | 2         | 2      | 2020-01-04 13:23:46 | 2020-01-04 13:53:03 | 00:29:17  | 40       | 0.59      | 3         |
| 104         | 5        | 3         | 4      | 2020-01-08 21:00:29 | 2020-01-08 21:10:57 | 00:10:28  | 15       | 0.67      | 1         |
| 105         | 7        | 2         | 4      | 2020-01-08 21:20:29 | 2020-01-08 21:30:45 | 00:10:16  | 25       | 1.00      | 1         |
| 102         | 8        | 2         | 5      | 2020-01-09 23:54:33 | 2020-01-10 00:15:02 | 00:20:29  | 15       | 1.56      | 1         |
| 104         | 10       | 1         | 3      | 2020-01-11 18:34:49 | 2020-01-11 18:50:20 | 00:15:31  | 10       | 1.00      | 2         |
| 104         | 10       | 1         | 3      | 2020-01-11 18:34:49 | 2020-01-11 18:50:20 | 00:15:31  | 10       | 1.00      | 2         |
#### 5. If a Meat Lovers pizza was \\$12 and Vegetarian \\$10 fixed prices with no cost for extras and each runner is paid \\$0.30 per kilometre traveled - how much money does Pizza Runner have left over after these deliveries?

In [ ]:
WITH prices as (
  	VALUES (1, 12), (2, 10)
)
SELECT '$' || SUM(price) - ROUND(SUM(distance * 0.3),2) AS profit
FROM (
  	SELECT
  		c.pizza_id,
  		distance
  	FROM pizza_runner.customer_orders c
  	JOIN pizza_runner.runner_orders r USING(order_id)
  	WHERE r.cancellation IS NULL
) AS sub
JOIN prices AS p(pizza_id,price) USING(pizza_id);

| profit |
| ------ |
| $73.38 |